In [1]:
from mpl_toolkits.basemap import Basemap
from pycm import ConfusionMatrix
from scipy import stats
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import dask.dataframe as dd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

sns.set()

In [2]:
app_events = dd.read_csv('talkingdata/app_events.csv')
app_labels = dd.read_csv('talkingdata/app_labels.csv')
events = dd.read_csv('talkingdata/events.csv')
train = dd.read_csv('talkingdata/gender_age_train.csv')
label_categories = dd.read_csv('talkingdata/label_categories.csv')
phone_brand_model = dd.read_csv('talkingdata/phone_brand_device_model.csv')

## Data wrangling

In [3]:
app_events.compute().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32473067 entries, 0 to 410969
Data columns (total 4 columns):
event_id        int64
app_id          int64
is_installed    int64
is_active       int64
dtypes: int64(4)
memory usage: 1.2 GB


In [4]:
app_labels.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459943 entries, 0 to 459942
Data columns (total 2 columns):
app_id      459943 non-null int64
label_id    459943 non-null int64
dtypes: int64(2)
memory usage: 7.0 MB


In [5]:
events.compute().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3252950 entries, 0 to 56838
Data columns (total 5 columns):
event_id     int64
device_id    int64
timestamp    object
longitude    float64
latitude     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 148.9+ MB


In [6]:
train.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 4 columns):
device_id    74645 non-null int64
gender       74645 non-null object
age          74645 non-null int64
group        74645 non-null object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [7]:
label_categories.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 2 columns):
label_id    930 non-null int64
category    927 non-null object
dtypes: int64(1), object(1)
memory usage: 14.6+ KB


In [8]:
phone_brand_model.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187245 entries, 0 to 187244
Data columns (total 3 columns):
device_id       187245 non-null int64
phone_brand     187245 non-null object
device_model    187245 non-null object
dtypes: int64(1), object(2)
memory usage: 4.3+ MB


No missing values except 3 in **label_categories**, which as of now will not be used in this project (but will be once I acquire more text mining skills).

---

It may be of interest to look at the number of apps each device has. Note:
- The **phone_brand_model** table has information regarding the brand and model of each device.
- The **events** table logs the device ID, time, and location of the device when a user uses the TalkingData SDK.
- The **app_events** contains information of all the apps installed on a device every time a user uses the TalkingData SDK.

The only way to get the total number of apps is to use the "events_id" from **events** as an intermediary. The steps are as follows:
- Get each unique "device_id" from **events** and *only* get the first occurrence of a "device_id" (to avoid double counting apps)
- Merge the above table with **app_events** to get the list of "app_id" for each device. The reason we only want the first occurrence of "device_id" in the above step is because every time a user uses an app that implements the TalkingData SDK, it creates an event in **events**. Each event corresponds to a list of apps in **app_events**, so if a user uses two apps that implements TalkingData's SDK, there will be two screenshots of the apps on the device in **app_events**. After getting the list of apps, group together by 'device_id' and count the number of apps.
- Merge the new table with **phone_brand_model** to get: "gender", "age", "group", "phone_brand", and "device_model"

In [9]:
# ---------------------------------------------------------------------
# CREATE NEW DATAFRAME THAT CONTAINS TOTAL NUMBER OF APPS PER DEVICE  |
# --------------------------------------------------------------------

num_apps = events.groupby('device_id').first().reset_index().compute()
num_apps = dd.merge(num_apps, app_events[['event_id', 'app_id']], on = 'event_id').compute()

num_apps = num_apps.groupby('device_id')['app_id'].count().reset_index()
num_apps = dd.merge(num_apps, phone_brand_model, on = 'device_id', how = 'inner') 

num_apps = num_apps.rename(columns={'app_id': 'app_count'})

num_apps.head()

,device_id,app_count,phone_brand,device_model
0,-9222956879900151005,68,三星,Galaxy Note 2
1,-9222661944218806987,10,vivo,Y913
2,-9222399302879214035,43,小米,MI 3
3,-9221767098072603291,25,金立,GN151
4,-9221079146476055829,12,小米,MI 3


In [10]:
# ------------------------------------------------------------------------------
# CREATE ANOTHER DATAFRAME WITH GENDER INFO (GET FROM TRAINING SET DATAFRAME)  |
# -----------------------------------------------------------------------------

gender_info = dd.merge(train, num_apps, on = 'device_id', how = 'inner')
gender_info = gender_info.rename(columns={'index': 'device_id'}).compute()

gender_info.head()

,device_id,gender,age,group,app_count,phone_brand,device_model
0,-8260683887967679142,M,35,M32-38,53,小米,MI 2
1,7477216237379271436,F,37,F33-42,26,华为,荣耀6 plus
2,6352067998666467520,M,32,M32-38,19,华为,荣耀畅玩4X
3,8026504930081700361,M,25,M23-26,31,小米,MI 4
4,-7271319853104672050,M,27,M27-28,34,三星,Galaxy Note 3
